In [2]:
#ライブラリをインクルード
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from tslearn.metrics import dtw_path
from datetime import datetime, timedelta
import os, re, csv, statistics, math
from decimal import Decimal
import bottleneck as bn
from scipy import signal, stats
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense
import japanize_matplotlib

2024-05-08 17:08:16.047788: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#Apple Watchからのモーションデータを読み込む関数
def process_apple_watch_csv(file_path):
    # CSVファイルの読み込み
    motion_data = pd.read_csv(file_path, header=0, names=['UnixTime', 'AccelerationX', 'AccelerationY', 'AccelerationZ', 'GyroX', 'GyroY', 'GyroZ'])
    # Unixタイムスタンプを日本時間に変換し、タイムゾーン情報を削除して表示する
    motion_data['Timestamp'] = pd.to_datetime(motion_data['UnixTime'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Tokyo').dt.tz_localize(None)
    # 加速度のユークリッドノルムを計算してデータフレームに追加
    motion_data['EuclideanNorm'] = np.sqrt(motion_data['AccelerationX']**2 + motion_data['AccelerationY']**2 + motion_data['AccelerationZ']**2)
    # FFTのサンプル数を取得
    N = len(motion_data)
    # サンプリングレートを定義 (50Hzとする)
    Fs = 50
    # FFT解析を行い、3軸加速度のパワースペクトルを計算してデータフレームに追加
    for axis in ['AccelerationX', 'AccelerationY', 'AccelerationZ', 'EuclideanNorm']:
        accel_fft = np.fft.fft(motion_data[axis])
        power_spectrum = np.abs(accel_fft)**2
        motion_data[f'PowerSpectrum_{axis}'] = power_spectrum

    return motion_data

In [ ]:
#Tobii Pro Glasses 2からのアイトラッキングデータを読み込む関数
def process_tobii_csv(file_path):
    # CSVファイルの読み込み
    eye_data = pd.read_csv(file_path, header=0)
    #タイムスタンプをYYYY-MM-DD HH:MM:SS.fff形式に変換
    eye_data['Recording start time'] = pd.to_datetime(eye_data['Recording start time'], format='%H:%M:%S.%f')
    eye_data['Recording date'] = pd.to_datetime(eye_data['Recording date'], format='%m/%d/%Y')
    eye_data['Recording timestamp'] = pd.to_datetime(eye_data['Recording timestamp'], unit='us')
    eye_data['Timestamp'] = eye_data['Recording date'] + pd.to_timedelta(eye_data['Recording start time'].dt.strftime('%H:%M:%S.%f')) + pd.to_timedelta(eye_data['Recording timestamp'].dt.strftime('%H:%M:%S.%f'))

    return eye_data[(eye_data['Sensor'] == 'Eye Tracker')].drop(columns=['Recording timestamp', 'Computer timestamp', 'Recording start time UTC', 'Recording duration', 'Recording Fixation filter name', 'Project name', 'Export date', 'Recording name','Recording date', 'Recording date UTC', 'Recording start time', 'Recording media name', 'Recording media width', 'Recording media height', 'Gyro X', 'Gyro Y', 'Gyro Z', 'Accelerometer X', 'Accelerometer Y', 'Accelerometer Z'])

In [ ]:
def process_watch_and_tobii_data(motion_data, eye_data):
    # データをマージ
    merged_data = pd.merge(eye_data, motion_data, on="Timestamp", how='outer')
    # eye_dataとmotion_dataのタイムスタンプ範囲を取得
    start_time = max(eye_data['Timestamp'].min(), motion_data['Timestamp'].min())
    end_time = min(eye_data['Timestamp'].max(), motion_data['Timestamp'].max())

    # マージ後のデータフレームを調整
    filtered_data = merged_data[(merged_data['Timestamp'] >= start_time) & (merged_data['Timestamp'] <= end_time)]

    return filtered_data

In [ ]:
def dist(x, y):
    return (x - y)**2

def get_min(m0, m1, m2, i, j):
    if m0 < m1:
        if m0 < m2:
            return i - 1, j, m0
        else:
            return i - 1, j - 1, m2
    else:
        if m1 < m2:
            return i, j - 1, m1
        else:
            return i - 1, j - 1, m2

In [ ]:
def spring(x, y, epsilon):
    Tx = len(x)
    Ty = len(y)

    C = np.zeros((Tx, Ty))
    B = np.zeros((Tx, Ty, 2), int)
    S = np.zeros((Tx, Ty), int)

    C[0, 0] = dist(x[0], y[0])

    for j in range(1, Ty):
        C[0, j] = C[0, j - 1] + dist(x[0], y[j])
        B[0, j] = [0, j - 1]
        S[0, j] = S[0, j - 1]

    for i in range(1, Tx):
        C[i, 0] = dist(x[i], y[0])
        B[i, 0] = [0, 0]
        S[i, 0] = i

        for j in range(1, Ty):
            pi, pj, m = get_min(C[i - 1, j],
                                C[i, j - 1],
                                C[i - 1, j - 1],
                                i, j)
            C[i, j] = dist(x[i], y[j]) + m
            B[i, j] = [pi, pj]
            S[i, j] = S[pi, pj]

        imin = np.argmin(C[:(i+1), -1])
        dmin = C[imin, -1]

        if dmin > epsilon:
            continue

        for j in range(1, Ty):
            if (C[i,j] < dmin) and (S[i, j] < imin):
                break
        else:
            path = [[imin, Ty - 1]]
            temp_i = imin
            temp_j = Ty - 1

            while (B[temp_i, temp_j][0] != 0 or B[temp_i, temp_j][1] != 0):
                path.append(B[temp_i, temp_j])
                temp_i, temp_j = B[temp_i, temp_j].astype(int)

            C[S <= imin] = 100000000
            yield np.array(path), dmin

In [ ]:
def plot_spring(data_x, data_y, timestamps, epsilon=0.1):
    pathes = []
    times = []

    for path, cost in spring(data_x, data_y, epsilon):
        plt.figure(figsize=(18, 6))  # グラフを横長にする
        for line in path:
            plt.plot(line, [data_x[line[0]], data_y[line[1]]], linewidth=0.8, c="gray")
        plt.plot(data_x, label='long data')
        plt.plot(data_y, label='gesuture data')


        plt.plot(path[:,0], data_x[path[:,0]], c="C2", label='simular')
        plt.grid(True)  # グリッド表示
        plt.legend()  # 凡例表示
        plt.xticks(rotation=45)  # x軸のラベルを45度回転

        plt.show()
        times.append(timestamps[path[:,0]])
        pathes.append(path)

    return pathes, times
#pathes, times = plot_spring(data2['EuclideanNorm'], circle2['EuclideanNorm'], data2['Timestamp'])

In [ ]:
# モデルの定義
model = Sequential()
model.add(LSTM(units=64, input_shape=(150, 6)))
model.add(Dense(units=5, activation='softmax'))

# モデルのコンパイル
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
motion_data20240504 = process_apple_watch_csv("datasets/kawano/20240504/1回目/MotionData_20240504_182506.csv")
eye_data20240504 = process_tobii_csv('datasets/kawano/20240504/1回目/kawano1回目.csv')
data20240504 = process_watch_and_tobii_data(motion_data20240504, eye_data20240504)

In [ ]:
data20240504

In [12]:
motion_data = process_apple_watch_csv("/Users/hinase/Downloads/MotionData_20240508_173037.csv")

In [13]:
motion_data['Timestamp'].diff().sort_values(ascending=0)

8223     0 days 00:00:00.024598016
36846    0 days 00:00:00.022172928
170666   0 days 00:00:00.021945088
110781   0 days 00:00:00.021890304
108503   0 days 00:00:00.021852160
                    ...           
5        0 days 00:00:00.018149888
110782   0 days 00:00:00.018121472
42633    0 days 00:00:00.018029824
8224     0 days 00:00:00.014931200
0                              NaT
Name: Timestamp, Length: 189631, dtype: timedelta64[ns]

In [11]:
motion_data['Timestamp'].diff().mean()

Timedelta('0 days 00:00:00.019989281')

In [ ]:
base_timestamps = [
    [pd.to_datetime('2024-05-04 18:30:00'), '0'],
    [pd.to_datetime('2024-05-04 18:31:00'), '1'],
    [pd.to_datetime('2024-05-04 18:32:00'), '2'],
    [pd.to_datetime('2024-05-04 18:33:00'), '3'],
    [pd.to_datetime('2024-05-04 18:34:00'), '5'],
    [pd.to_datetime('2024-05-04 18:35:00'), '4'],
    [pd.to_datetime('2024-05-04 18:36:00'), '6'],
    [pd.to_datetime('2024-05-04 18:37:00'), '7'],
    [pd.to_datetime('2024-05-04 18:38:00'), '8'],
    [pd.to_datetime('2024-05-04 18:39:00'), '9'],
    [pd.to_datetime('2024-05-04 18:44:00'), '0'],
    [pd.to_datetime('2024-05-04 18:44:30'), '1'],
    [pd.to_datetime('2024-05-04 18:45:00'), '2'],
    [pd.to_datetime('2024-05-04 18:45:30'), '3'],
    [pd.to_datetime('2024-05-04 18:46:00'), '4'],
    [pd.to_datetime('2024-05-04 18:46:35'), '4'],
    [pd.to_datetime('2024-05-04 18:47:00'), '5'],
    [pd.to_datetime('2024-05-04 18:47:30'), '6'],
    [pd.to_datetime('2024-05-04 18:48:00'), '7'],
    [pd.to_datetime('2024-05-04 18:48:30'), '8'],
    [pd.to_datetime('2024-05-04 18:49:00'), '9'],
]

In [ ]:
# intervals リストを計算
intervals = [(timestamp + pd.Timedelta(seconds=-2), timestamp + pd.Timedelta(seconds=3), label) for timestamp, label in base_timestamps]

# 結果の表示
for start, end, label in intervals:
    print(f"Start: {start}, End: {end}, Label: {label}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

# プロットの初期設定
plt.figure(figsize=(60, 10))

# グレーで全データをプロット
plt.plot(data20240504['Timestamp'], data20240504['Pupil diameter filtered'], color='gray', label='Pupil Diameter Filtered') #瞳孔径
plt.plot(data20240504['Timestamp'], data20240504['EuclideanNorm'], color='gray', label='Euclidean Norm')    #3軸合成加速度

# intervals リストを使用して特定の区間を色付きでハイライト
colors = ['red', 'blue', 'green', 'orange', 'purple']  # カラーリスト
for i, (start, end, label) in enumerate(intervals):
    # 指定区間のデータを抽出
    highlight_data = data20240504[(data20240504['Timestamp'] >= start) & (data20240504['Timestamp'] <= end)]

    # 特定区間をプロット
    plt.plot(highlight_data['Timestamp'], highlight_data['Pupil diameter filtered'], color=colors[i % len(colors)], label=f'Pupil Dia: {label}')
    plt.plot(highlight_data['Timestamp'], highlight_data['EuclideanNorm'], color=colors[i % len(colors)], label=f'Euclidean Norm: {label}')

# グラフの装飾
plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.title('Highlighted Intervals in Pupil Diameter and Motion Data')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
plt.tight_layout()

# プロットを表示
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

# プロットの初期設定
plt.figure(figsize=(60, 10))

# Pupil diameter filtered の差分の二乗を計算
diff_squared = data20240504['Pupil diameter filtered'].diff()**2

# グレーで全データをプロット
plt.plot(data20240504['Timestamp'], diff_squared * -1, color='gray', label='Squared Diff of Pupil Diameter (Inverted)')  # 瞳孔径の差分の二乗（逆転表示）
plt.plot(data20240504['Timestamp'], data20240504['EuclideanNorm'], color='gray', label='Euclidean Norm')  # 3軸合成加速度

# intervals リストを使用して特定の区間を色付きでハイライト
colors = ['red', 'blue', 'green', 'orange', 'purple']  # カラーリスト
for i, (start, end, label) in enumerate(intervals):
    # 指定区間のデータを抽出
    highlight_data = data20240504[(data20240504['Timestamp'] >= start) & (data20240504['Timestamp'] <= end)]

    # 特定区間をプロット
    plt.plot(highlight_data['Timestamp'], highlight_data['Pupil diameter filtered'].diff()**2 * -1, color=colors[i % len(colors)], label=f'Sq Diff Pupil Dia (Inverted): {label}')
    plt.plot(highlight_data['Timestamp'], highlight_data['EuclideanNorm'], color=colors[i % len(colors)], label=f'Euclidean Norm: {label}')

# グラフの装飾
plt.xlabel('Timestamp')
plt.ylabel('Values (Inverted for Pupil Diameter Diff Squared)')
plt.title('Highlighted Intervals in Pupil Diameter and Motion Data')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
plt.tight_layout()

# プロットを表示
plt.show()


In [ ]:
# プロットの初期設定
plt.figure(figsize=(160, 10))

# 注視点の移動距離を計算
data20240504['idouryou'] = np.sqrt(data20240504['Gaze point X'].diff()**2 + data20240504['Gaze point Y'].diff()**2)

# グレーで全データをプロット
plt.plot(data20240504['Timestamp'], data20240504['idouryou'], color='gray', label='Inverted - Gaze Movement Distance')  # 注視点の移動距離（逆転表示）
#plt.plot(data20240504['Timestamp'], data20240504['EuclideanNorm'], color='gray', label='Euclidean Norm')  # 3軸合成加速度

# intervals リストを使用して特定の区間を色付きでハイライト
colors = ['red', 'blue', 'green', 'orange', 'purple']  # カラーリスト
for i, (start, end, label) in enumerate(intervals):
    # 指定区間のデータを抽出
    highlight_data = data20240504[(data20240504['Timestamp'] >= start) & (data20240504['Timestamp'] <= end)]

    # 特定区間をプロット
    plt.plot(highlight_data['Timestamp'], highlight_data['idouryou'], color=colors[i % len(colors)], label=f'Inverted Gaze Move: {label}')
    #plt.plot(highlight_data['Timestamp'], highlight_data['EuclideanNorm'], color=colors[i % len(colors)], label=f'Euclidean Norm: {label}')

# グラフの装飾
plt.xlabel('Timestamp')
plt.ylabel('Gaze Movement Distance (downward)')
plt.title('Highlighted Intervals in Gaze Movement')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
plt.tight_layout()

# プロットを表示
plt.show()
